In [ ]:
# default_exp layers
%load_ext nb_black

<IPython.core.display.Javascript object>

# `layers`

> Blaze CNN Layers

In [ ]:
# export

import torch.nn as nn
import torch.nn.functional as F

class BlazeBlock(nn.Module):
    """Based on `https://github.com/hollance/BlazeFace-PyTorch/blob/master/blazeface.py`"""
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1, use_res_conv=False):
        super().__init__()
        
        self.input_pad_units = (1, 2, 1, 2) if kernel_size == 5 else (0, 2, 0, 2)
        if stride == 2:
            padding = 0
        else:
            padding = (kernel_size - 1) // 2
        self.stride = stride
        self.channel_pad =  out_channels - in_channels
        
        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels, 
                in_channels, 
                kernel_size=kernel_size, 
                stride=stride, 
                padding=padding, 
                groups=in_channels
            ),
            nn.Conv2d(in_channels, out_channels, 1)
        )
        if self.stride == 2:
            if not use_res_conv:
                self.short =  nn.MaxPool2d(kernel_size=stride, stride=stride)
            else:
                self.short = nn.Sequential(
                    nn.MaxPool2d(kernel_size=stride, stride=stride),
                    nn.Conv2d(in_channels, out_channels, 1)
                )
                self.channel_pad = 0
        
        self.activation = nn.ReLU(inplace=True)
        
    def forward(self, x):

        if self.stride == 2:
            h = F.pad(x, self.input_pad_units, "constant", 0)
            x = self.short(x)
        else:
            h = x

        if self.channel_pad > 0:
            x = F.pad(x, (0, 0, 0, 0, 0, self.channel_pad), "constant", 0)

  
        return self.activation(self.conv(h) + x)


<IPython.core.display.Javascript object>

In [ ]:
import torch

x = torch.randn(1, 8, 10, 10)
block = BlazeBlock(8, 16, kernel_size=5, stride=2)
block(x).shape

torch.Size([1, 16, 5, 5])

<IPython.core.display.Javascript object>

In [ ]:
# export
class BlazeFaceBlock(BlazeBlock):
    """Basic Block for BlazeFace Model"""

    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
        super().__init__(
            in_channels, out_channels, kernel_size, stride, use_res_conv=False
        )

<IPython.core.display.Javascript object>

In [ ]:
x = torch.randn(1, 8, 10, 10)
block = BlazeFaceBlock(8, 16, stride=2)
assert tuple(block(x).shape) == (1, 16, 5, 5)

<IPython.core.display.Javascript object>

In [ ]:
x = torch.randn(1, 8, 10, 10)
block = BlazeFaceBlock(8, 16, stride=1)
assert tuple(block(x).shape) == (1, 16, 10, 10)

<IPython.core.display.Javascript object>

In [ ]:
# export
class BlazePoseBlock(BlazeBlock):
    """Basic Block for BlazePose Model"""

    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1):
        super().__init__(
            in_channels, out_channels, kernel_size, stride, use_res_conv=True
        )

<IPython.core.display.Javascript object>

In [ ]:
x = torch.randn(1, 8, 10, 10)
block = BlazePoseBlock(8, 16, stride=1)
assert tuple(block(x).shape) == (1, 16, 10, 10)

<IPython.core.display.Javascript object>

In [ ]:
x = torch.randn(1, 8, 10, 10)
block = BlazePoseBlock(8, 16, stride=2)
assert tuple(block(x).shape) == (1, 16, 5, 5)

<IPython.core.display.Javascript object>